In [1]:
import math
import numpy as np
from datetime import datetime

import skyfield
from skyfield.api import load
from skyfield.api import N, W, S, E
from skyfield.api import Star
from skyfield.data import hipparcos

ts = load.timescale()
planets = load('de421.bsp')
earth = planets['earth']

with load.open(hipparcos.URL) as f:
    df = hipparcos.load_dataframe(f)

In [2]:
# Mind blown... python uses Bakers Rounding.  Got to resort to this to get proper rounding.
from decimal import *
getcontext().rounding = ROUND_HALF_UP

def normal_round(value, precision):
    value = value * 10**precision
    value = Decimal(value).to_integral_value()
    value = value / 10**precision
    return float(value)

In [3]:
def createAngle(degrees, minutes, sign):
    if (sign == None): 
        sign = 1
    return (degrees + minutes/60.0) * sign

def toDegreesAndMinutes(angle):
    sign = 1
    if (angle < 0): sign = -1
    angle = angle*sign
    degrees = math.floor(angle)
    minutes = normal_round((angle - degrees) * 60.0, 2)
    
    return degrees*sign, minutes
    
def angleToString(angle):
    degrees, minutes = toDegreesAndMinutes(angle)
    return "{} degrees; {} minutes".format(degrees, minutes)

def angleToStringDelta(angle):
    degrees, minutes = toDegreesAndMinutes(angle)
    if (degrees == 0):
        return "{} minutes".format(minutes)
    else:
        return "{} degrees; {} minutes".format(degrees, minutes)
    
def diffAngle (angle1, angle2, zero_threshold):
    threesixty_threshold = 360-zero_threshold
    
    if ((angle1 < zero_threshold) and (angle2 > threesixty_threshold)):
        angle1 += 360 # Move up Angle 1
    elif((angle2 < zero_threshold) and (angle1 > threesixty_threshold)):
        angle2 += 360 # Move up Angle 2
    
    return abs (angle1-angle2)
    

In [4]:
def compute_GHA_dec(celestial_obj, utc): 
    dt = datetime.strptime(utc, '%Y/%m/%d %H:%M:%S')
    t = ts.ut1(dt.year, dt.month, dt.day, dt.hour, dt.minute, dt.second)
    position = earth.at(t).observe(celestial_obj)
    ra = position.apparent().radec(epoch='date')[0]
    dec = position.apparent().radec(epoch='date')[1]
    distance = position.apparent().distance()
    
    gha = (t.gast-ra.hours)*15
    
    if (gha < 0):
        gha += 360

    return gha, dec.degrees, distance.au    

In [5]:
def compute_LHA(GHA, dr_lon):
    # Compute LHA.
    if (dr_lon < 0):
        # Western hemisphere.  LHA = GHA - a_lon
        LHA = GHA - (dr_lon*W)
    else:
        # Easter hemisphere.  LHA = GHA + a_lon
        LHA = GHA + (dr_lon*E)
        
    if (LHA < 0):
        LHA += 360.0
    elif (LHA > 360.0):
        LHA -= 360.0
    return LHA

In [6]:
def sight_reduction(dec, lat, LHA):
    dec_rads = math.radians(dec)
    lat_rads = math.radians(lat)
    LHA_rads = math.radians(LHA)
    
    if ((dec_rads < 0) and (lat_rads < 0)):
        # If dec and lat are same hemisphere (i.e. SAME), then both values should be positive.
        # here there are both south (so negative) take the absolute value so that they are both positive.
        dec_rads = abs(dec_rads)
        lat_rads = abs(lat_rads)
    elif ((dec_rads > 0) and (lat_rads < 0)):
        # if dec and lat different hemispheres (i.e. CONTRARY), then dec should be made negative.
        # here we have a south latitude and north declination, so shift the negative to dec.
        dec_rads = dec_rads *-1
        lat_rads = lat_rads *-1
    
    # Hc
    sin_Hc_rads = math.sin(lat_rads)*math.sin(dec_rads) + math.cos(lat_rads)*math.cos(dec_rads)*math.cos(LHA_rads) 
    Hc_rads = math.asin(sin_Hc_rads)
    Hc = math.degrees(Hc_rads)
    
    #Z       
    cos_Z_rads_num = math.sin(dec_rads) - math.sin(Hc_rads)*math.sin(lat_rads)
    cos_Z_rads_den = math.cos(Hc_rads) * math.cos(lat_rads) 
    cos_Z_rads = cos_Z_rads_num / cos_Z_rads_den
    
    # Clip values that are > 1 or < -1  (usually introduced due to rounding error)
    if (cos_Z_rads > 1):
        print("cos_Z_rads > 1 {}".format(cos_Z_rads))
        cos_Z_rads = 1
    elif (cos_Z_rads < -1):
        print("cos_Z_rads < 1 {}".format(cos_Z_rads))
        cos_Z_rads = -1
    
    Z = math.degrees(math.acos(cos_Z_rads))
    
    return Hc, Z

## Altitude Correction
There are three adjustments for altitude:
1. Atmospheric Refraction (applicable to all bodies)
2. Horizontal Parallax (Most applicable to the Moon, and the Sun in some instances.)
3. Semi-Diameter (applicable to Sun, Moon primarily but potentially Venus and Mars)

### Computing Refraction
$$
\frac{1}{tan(Ha + \frac{7.31}{Ha+4.4})}
$$

### Computing Horizontal Parallax (HP)
$$
HP = arcsin(\frac{Earth's Radius (Equitorial)}{Distance of Body (km)})
$$

HP is then corrected for altitude as effect of Horizontal Parallax is less with higher altitude (none at 90 degrees)

$$
HP(altitude) = HP x cos(Ha)
$$

HP-alt should be added to Ha

### Computing Semi-Diameter (SD)
When measuring the altitudes of the Sun, Moon, Venus and Mars, it is usual to use either the top (Upper Limb) or bottom (Lower Limb) of the body.  This offset must then be removed before comparision with the calculated value.

The angular diameter of a body depends on its distance from the Earth.  Thus for the Sun, the Semi-Diameter varies between 16'.3 in January, when the Sun is closest and 15'.7 in June when it is furthest away.  

The formula for the angular diameter $\alpha$ of an object with a diameter D at a distance r is:

$$
\alpha = 2 arctan (\frac{D}{2r})
$$

For Semi-Diameter (half of the angular diameter) this equation simplifies to:

$$
SD = arctan(\frac{object radius}{distance (km)})
$$


Lower Limb should be added to to the true altitude (Ha), Upper Limb should be subtracted.

In [7]:
Off = 1
On = -1

LowerLimb_Sun = 0
UpperLimb_Sun = 1
LowerLimb_Moon = 2
UpperLimb_Moon = 3

sun_equatorial_radius = 696000 #km
moon_equatorial_radius = 1738.1 #km
earth_equatorial_radius = 6378.1370 #km
au_to_km = 149597870

sd_moon_correction = 0.2724

def compute_dip_correction(heightEyeFt):
    dip = 0.97*math.sqrt(heightEyeFt)
    dip = normal_round(dip, 1)
    dip *= -1
    return dip

def compute_altitude_correction(Ha, distance, limb, lat):
    distance *= au_to_km # convert from AU (Astronomical Units to Kilometers)
    
    # First compute Refraction - will be the same for all bodies.
    # Round refraction to tenth of minutes... always negative.
    refraction = 1.0/ math.tan(math.radians(Ha + (7.31/(Ha+4.4))))
    refraction = normal_round(refraction, 1)
    refraction *= -1
    
    # Second compute HP (horizontal parallax)
    # adjust the impact of HP by the cos of Ha
    # Round HP to tenth of minutes... always positive.
    hp = math.degrees(math.asin(earth_equatorial_radius/distance))         
    hp = hp * math.cos(math.radians(Ha))  
    hp_minutes = normal_round(hp * 60, 1)

    # Third compute SD (Semi-diameter).
    sd = 0
    celestial_body_radius = 0
    moon_correction = False
    if ((limb == LowerLimb_Sun) or (limb == UpperLimb_Sun)):
        celestial_body_radius = sun_equatorial_radius
    elif ((limb == LowerLimb_Moon) or (limb == UpperLimb_Moon)):
        celestial_body_radius = moon_equatorial_radius
        moon_correction = True
    sd = math.degrees(math.asin(celestial_body_radius / distance))
    
    # Round SD to tenth of minutes... if Upper Limb the value is negative.
    sd_minutes = normal_round(sd * 60, 1)
    if ((limb == UpperLimb_Sun) or (limb == UpperLimb_Moon)):
        sd_minutes *= -1 
    
    return refraction, sd_minutes, hp_minutes


In [8]:
def compute_Zn(Z, lat, LHA):
    #Zn
    Zn = Z
    if (lat > 0):
        # Northern latitude.
        if (LHA < 180):
            Zn = 360 - Z
    else:
        # Souther latitude.
        if (LHA > 180):
            Zn = 180-Z
        else:
            Zn = 180+Z
            
    if (Zn == 360):
        Zn = 0
    return Zn

In [9]:
def compute_a(Ho, Hc):
    if (Ho > Hc):
        return Ho - Hc, 'T'
    if (Ho < Hc):
        return Hc - Ho, 'A'

In [10]:
def compute_lat_from_Ho(dec, Ho, dr_lat):
    looking = N
    if (dr_lat > 23.5):
        looking = S
    elif (dr_lat < -23.5):
        looking = N
    elif (dec < dr_lat):
        looking = S
    else:
        looking = N
    
    zenith = 90 - Ho
    return dec - zenith*looking

In [11]:
def compute_position(celestial_obj_string, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt):
    #print("utc: {}".format(utc))
    #print("dr_lat: {}".format(angleToString(dr_lat)))
    #print("dr_lon: {}".format(angleToString(dr_lon)))
    #print(celestial_obj_string)
    
    dt = datetime.strptime(utc, '%Y/%m/%d %H:%M:%S')
    celestial_obj = None
    limb = None
    
    if (celestial_obj_string == "Sun-LL"):
        celestial_obj = planets['Sun']
        limb = LowerLimb_Sun
    elif (celestial_obj_string == "Sun-UL"):
        celestial_obj = planets['Sun']
        limb = UpperLimb_Sun
    elif (celestial_obj_string == "Moon-LL"):
        celestial_obj = planets['Moon']
        limb = LowerLimb_Moon
    elif (celestial_obj_string == "Moon-UL"):
        celestial_obj = planets['Moon']
        limb = UpperLimb_Moon
    elif (celestial_obj_string == "Venus"):
        celestial_obj = planets['Venus']
    elif (celestial_obj_string == "Jupiter"):
        celestial_obj = planets['JUPITER BARYCENTER']
    elif (celestial_obj_string == "Altair"):
        celestial_obj = Star.from_dataframe(df.loc[97649])
    elif (celestial_obj_string == "Arcturus"):
        celestial_obj = Star.from_dataframe(df.loc[69673])
    elif (celestial_obj_string == "Antares"):
        celestial_obj = Star.from_dataframe(df.loc[80763])
    elif (celestial_obj_string == "Regulus"):
        celestial_obj = Star.from_dataframe(df.loc[49669])
    elif (celestial_obj_string == "Hamal"):
        celestial_obj = Star.from_dataframe(df.loc[9884])
    elif (celestial_obj_string == "Sirius"):
        celestial_obj = Star.from_dataframe(df.loc[32349])
    
    GHA, dec, distance = compute_GHA_dec(celestial_obj, utc)
    
    #print("GHA: {}".format(angleToString(GHA)))
    #print("dec: {}".format(angleToString(dec)))
    #print("distance: {}".format(distance))
    
    #print("Hs: {}".format(angleToString(Hs)))
    #print("ic: {}".format(angleToString(ic)))
    dip = compute_dip_correction(heightEyeFt) 
    #print("heightEyeFt: {}; dip (minutes): {}".format(heightEyeFt, dip)) 
    
    Ha = Hs + ic + createAngle(0, dip, None)
    #print("Ha: {}".format(angleToString(Ha)))
    
    refraction, sd_minutes, hp_minutes = compute_altitude_correction(Ha, distance, limb, dr_lat)   
    #print("SD (minutes): {}".format(sd_minutes))
    #print("refraction (minutes): {}".format(refraction))
    #print("HP (minutes): {}".format(hp_minutes))
    
    Ho = Ha + (sd_minutes + refraction + hp_minutes) / 60.0
    #print("Ho: {}".format(angleToString(Ho)))

    LHA = compute_LHA(dr_lon, GHA)
    #print("LHA: {}".format(angleToString(LHA)))

    Hc, Z = sight_reduction(dec, dr_lat, LHA)
    Zn = compute_Zn(Z, dr_lat, LHA)
    #print("Hc: {}".format(angleToString(Hc)))
    #print("Z: {}".format(angleToString(Z)))
    #print("Zn: {}".format(angleToString(Zn)))

    a, ta = compute_a(Ho, Hc)
    #print("Azimuth: {} {}; Intercept: {}".format(angleToString(Zn), ta, angleToString(a)))
    #print("a: {} {} {}".format(angleToString(a), ta, angleToString(Zn)))
    
    return Ho, LHA, Hc, Zn, a, ta

In [12]:
def calculate_fix(a_lat, a_lon, lop1_Zn, lop1_ta, lop1_a, lop2_Zn, lop2_ta, lop2_a):
    if (lop1_ta == "A"):
        lop1_Zn+=180
        if (lop1_Zn > 360):
            lop1_Zn-=360
            
    if (lop2_ta == "A"):
        lop2_Zn+=180
        if (lop2_Zn > 360):
            lop2_Zn-=360
    
    a1 = lop1_a/math.cos(math.radians(lop1_Zn))
    a2 = lop2_a/math.cos(math.radians(lop2_Zn))
    b1 = -math.tan(math.radians(lop1_Zn))
    b2 = -math.tan(math.radians(lop2_Zn))
    
    d_lat = (a1+(a1-a2)/(b2/b1-1))
    lat_fix = a_lat + d_lat
    
    d_lon = (a1-a2)/(b2-b1)/math.cos(math.radians(lat_fix))
    lon_fix = a_lon+d_lon
    
    if (lon_fix > 180):
        lon_fix -= 360
        
    if (lon_fix <-180):
        lon_fix +=360
        
    return lat_fix, lon_fix

Cell for verification tests in changing any of the above logic.

In [13]:
test_data = [
    # Sun Data
    ["Sun-LL", "1978/7/25 23:4:0", createAngle(56, 2, N), createAngle(164, 30, W), createAngle(54, 5, None), createAngle(0, 2, On), 9],
    ["Sun-UL", "1978/10/27 22:49:0", createAngle(10, 0, S), createAngle(166, 15, W), createAngle(87, 20, None), createAngle(0, 2, Off), 25],
    ["Sun-LL", "1978/10/25 6:48:0", createAngle(35, 54, N), createAngle(74, 2, E), createAngle(41, 30.5, None), createAngle(0, 4, Off), 10],
    ["Sun-LL", "1981/3/27 13:7:0", createAngle(37, 40, N), createAngle(15, 24, W), createAngle(54, 41.3, None), createAngle(0, 2, Off), 9],
    ["Sun-LL", "1981/3/28 7:48:0", createAngle(22, 38, N), createAngle(64, 17, E), createAngle(71, 9.8, None), createAngle(0, 2.2, On), 9],
    ["Sun-LL", "1978/10/26 0:51:0", createAngle(27, 6, S), createAngle(163, 16, E), createAngle(74, 59.8, None), createAngle(0, 1.4, Off), 9],
    ["Sun-UL", "1978/7/24 17:48:0", createAngle(28, 44, S), createAngle(85, 17, W), createAngle(42, 51.2, None), createAngle(0, 1.5, On), 9],
    ["Sun-LL", "1978/7/25 20:50:0", createAngle(44, 10, N), createAngle(131, 0, W), createAngle(65, 22.5, None), createAngle(0, 2, On), 9],
    ["Sun-LL", "1981/3/28 21:51:0", createAngle(16, 40, S), createAngle(146, 30, W), createAngle(69, 44.2, None), createAngle(0, 2.5, Off), 12],
    ["Sun-LL", "1978/10/25 10:4:0", createAngle(34, 29, N), createAngle(25, 0, E), createAngle(43, 11.7, None), createAngle(0, 1.5, Off), 14],
    
    # Star Data
    ["Altair", "1978/7/26 4:51:30", createAngle(45, 30, N), createAngle(126, 27, W), createAngle(35, 18.9, None), createAngle(0, 0, Off), 12],
    ["Arcturus", "1978/7/25 3:49:3", createAngle(45, 30, N), createAngle(120, 58, W), createAngle(57, 57.4, None), createAngle(0, 1.8, On), 16],
    ["Altair", "1978/7/25 4:7:22", createAngle(44, 36, N), createAngle(122, 14, W), createAngle(30, 35.4, None), createAngle(0, 2, Off), 16],
    ["Antares", "1978/7/25 4:7:22", createAngle(44, 36, N), createAngle(122, 14, W), createAngle(18, 54.3, None), createAngle(0, 2, Off), 16],
    ["Arcturus", "1978/7/26 4:48:17", createAngle(45, 30, N), createAngle(126, 27, W), createAngle(50, 50.9, None), createAngle(0, 0, Off), 12],
    ["Regulus", "1981/3/28 3:49:5", createAngle(45, 21, N), createAngle(130, 3, W), createAngle(42, 58.5, None), createAngle(0, 1.2, Off), 9],
    ["Hamal", "1978/10/25 18:48:39", createAngle(44, 5, N), createAngle(160, 25, E), createAngle(19, 58.3, None), createAngle(0, 2.5, On), 19],
    ["Sirius", "1978/7/25 18:49:21", createAngle(45, 30, S), createAngle(33, 40, W), createAngle(11, 5.2, None), createAngle(0, 0.5, Off), 15],

    # Planet Data
    ["Venus", "1978/10/27 17:50:15", createAngle(44, 50, S), createAngle(15, 10, E), createAngle(16, 12.6, None), createAngle(0, 1.5, Off), 9],
    ["Jupiter", "1978/10/25 19:4:28", createAngle(44, 5, N), createAngle(160, 25, E), createAngle(62, 51.9, None), createAngle(0, 2.5, On), 19],
    ["Venus", "1978/7/25 19:5:2", createAngle(45, 30, S), createAngle(33, 40, W), createAngle(31, 55.6, None), createAngle(0, 0.5, Off), 15],

    # Moon Data
    ["Moon-UL", "1978/10/25 6:51:10", createAngle(44, 50, N), createAngle(40, 20, W), createAngle(42, 38.1, None), createAngle(0, 0, Off), 10],
    ["Moon-UL", "1978/7/26 14:49:54", createAngle(44, 58, N), createAngle(122, 24, W), createAngle(51, 25.2, None), createAngle(0, 0, Off), 16],
    ["Moon-UL", "1978/10/25 19:7:2", createAngle(45, 5, N), createAngle(160, 25, E), createAngle(51, 42.9, None), createAngle(0, 2.5, On), 19],
    ["Moon-LL", "1981/3/27 14:49:29", createAngle(45, 16, N), createAngle(140, 20, W), createAngle(24, 49.7, None), createAngle(0, 0.1, On), 10]
    
]

expected = [
    # Sun Expected
    [createAngle(359, 53.3, None), createAngle(54, 15.2, None), createAngle(53, 32.4, None), createAngle(0, 42.8, None), "T", 179.8],
    [createAngle(0, 1.6, None), createAngle(87, 1, None), createAngle(87, 5.5, None), createAngle(0, 4.6, None), "A", 180.5],
    [createAngle(359, 59.5, None), createAngle(41, 46.5, None), createAngle(42, 6.2, None), createAngle(0, 19.7, None), "A", 180],
    [createAngle(0, 0.3, None), createAngle(54, 55.8, None), createAngle(55, 1.4, None), createAngle(0, 5.6, None), "A", 180],
    [createAngle(359, 59.9, None), createAngle(71, 20.4, None), createAngle(70, 21.6, None), createAngle(0, 58.8, None), "T", 180],
    [createAngle(359, 59.8, None), createAngle(75, 14.1, None), createAngle(75, 9.3, None), createAngle(0, 4.8, None), "T", 0],
    [createAngle(0, 6.5, None), createAngle(42, 30.1, None), createAngle(41, 26, None), createAngle(1, 4.1, None), "T", 359.9],
    [createAngle(359, 53.3, None), createAngle(65, 32.9, None), createAngle(65, 25.6, None), createAngle(0, 7.4, None), "T", 179.7],
    [createAngle(0, 0.6, None), createAngle(69, 59.0, None), createAngle(70, 6.7, None), createAngle(0, 7.6, None), "A", 0],
    [createAngle(359, 57.8, None), createAngle(43, 24.7, None), createAngle(43, 28.4, None), createAngle(0, 3.7, None), "A", 180],   
    
    # Star Expected
    [createAngle(312, 31.1, None), createAngle(35, 14.1, None), createAngle(35, 16.1, None), createAngle(0, 2, None), "A", 116.9],
    [createAngle(25, 7.9, None), createAngle(57, 51.1, None), createAngle(56, 34.6, None), createAngle(1, 16.5, None), "T", 226.7],
    [createAngle(304, 41.2, None), createAngle(30, 31.8, None), createAngle(30, 31.9, None), createAngle(0, .1, None), "A", 109.4],
    [createAngle(355, 6, None), createAngle(18, 49.5, None), createAngle(18, 52.5, None), createAngle(0, 3, None), "A", 175.4],
    [createAngle(35, 29, None), createAngle(50, 46.7, None), createAngle(50, 45.0, None), createAngle(0, 1.7, None), "T", 240],
    [createAngle(320, 51.4, None), createAngle(42, 55.7, None), createAngle(42, 58.3, None), createAngle(0, 2.6, None), "A", 122.5],
    [createAngle(84, 52.8, None), createAngle(19, 48.8, None), createAngle(19, 33.4, None), createAngle(0, 15.4, None), "T", 284],
    [createAngle(90, 44.8, None), createAngle(10, 57, None), createAngle(11, 18.4, None), createAngle(0, 21.3, None), "A", 257.6],
    
    # Planet Expected
    [createAngle(91, 46.2, None), createAngle(16, 8.3, None), createAngle(15, 37.1, None), createAngle(0, 31.2, None), "T", 251.2],
    [createAngle(350, 18.1, None), createAngle(62, 44.7, None), createAngle(63, 23.3, None), createAngle(0, 38.7, None), "A", 159.1],
    [createAngle(28, 49.1, None), createAngle(31, 50.9, None), createAngle(31, 58.9, None), createAngle(0, 8.0, None), "A", 325.6],

    # Moon Expected
    [createAngle(319, 7.6, None), createAngle(42, 59.3, None), createAngle(42, 55.7, None), createAngle(0, 3.5, None), "T", 119.4],
    [createAngle(15, 40.8, None), createAngle(51, 40.5, None), createAngle(51, 23, None), createAngle(0, 17.6, None), "T", 205.3],
    [createAngle(338, 17.1, None), createAngle(51, 54.5, None), createAngle(51, 41.3, None), createAngle(0, 13.1, None), "T", 144.2],
    [createAngle(3, 58.5, None), createAngle(25, 49.2, None), createAngle(24, 59.8, None), createAngle(0, 49.4, None), "T", 184.1]
]

threshold = createAngle(0, 0.3, None)
Zn_threshold = createAngle(1, 0, None)  # Answers from book use a method of rounding to nearest degree.

success = True

for i in range (len(test_data)):
    celestial_object, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt = test_data[i]
    Ho, LHA, Hc, Zn, a, ta = compute_position (celestial_object, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt)
    LHA_expected, Ho_expected, Hc_expected, a_expected, ta_expected, Zn_expected = expected[i]
    
    print ("SIGHTING for {} at {}".format(celestial_object, utc))
    
    LHA_delta = diffAngle(LHA, LHA_expected, threshold)
    LHA_passed = (LHA_delta<threshold)
    print ("{}: LHA={}; delta={}".format("PASSED" if LHA_passed else "FAILED", angleToString(LHA), angleToStringDelta(LHA_delta)))
    
    Ho_delta = abs(Ho-Ho_expected)
    Ho_passed = (Ho_delta<threshold)
    print ("{}: Ho={}; delta={}".format("PASSED" if Ho_passed else "FAILED", angleToString(Ho), angleToStringDelta(Ho_delta)))
    
    Hc_delta = abs(Hc-Hc_expected)
    Hc_passed = (Hc_delta<threshold)
    print ("{}: Hc={}; delta={}".format("PASSED" if Hc_passed else "FAILED", angleToString(Hc), angleToStringDelta(Hc_delta)))
    
    a_delta = abs(a-a_expected)
    a_passed = ((a_delta<threshold) and (ta == ta_expected))
    print ("{}: a={} {}; delta={}".format("PASSED" if a_passed else "FAILED", angleToString(a), ta, angleToStringDelta(a_delta)))
    if (ta != ta_expected):
        print("MISMATCH on T/A:  ta={}; ta_expected={}".format(ta, ta_expected))
   
    Zn_delta = diffAngle(Zn, Zn_expected, Zn_threshold)
    Zn_passed = (Zn_delta<Zn_threshold)
    print ("{}: Zn={}; delta={}".format("PASSED" if Zn_passed else "FAILED", angleToString(Zn), angleToStringDelta(Zn_delta)))
    
    print ("==========================")
    
    if (success == True):
        success = LHA_passed and Ho_passed and Hc_passed and a_passed and Zn_passed

    
print("Overall result: {}".format("PASSED" if (success==True) else "FAILED! Check above results."))



SIGHTING for Sun-LL at 1978/7/25 23:4:0
PASSED: LHA=359 degrees; 53.32 minutes; delta=0.02 minutes
PASSED: Ho=54 degrees; 15.2 minutes; delta=0.0 minutes
PASSED: Hc=53 degrees; 32.35 minutes; delta=0.05 minutes
PASSED: a=0 degrees; 42.85 minutes T; delta=0.05 minutes
PASSED: Zn=179 degrees; 49.41 minutes; delta=1.41 minutes
SIGHTING for Sun-UL at 1978/10/27 22:49:0
PASSED: LHA=0 degrees; 1.65 minutes; delta=0.05 minutes
PASSED: Ho=87 degrees; 1.0 minutes; delta=0.0 minutes
PASSED: Hc=87 degrees; 5.55 minutes; delta=0.05 minutes
PASSED: a=0 degrees; 4.55 minutes A; delta=0.05 minutes
PASSED: Zn=180 degrees; 31.67 minutes; delta=1.67 minutes
SIGHTING for Sun-LL at 1978/10/25 6:48:0
PASSED: LHA=359 degrees; 59.54 minutes; delta=0.04 minutes
PASSED: Ho=41 degrees; 46.5 minutes; delta=0.0 minutes
PASSED: Hc=42 degrees; 6.2 minutes; delta=0.0 minutes
PASSED: a=0 degrees; 19.7 minutes A; delta=0.0 minutes
PASSED: Zn=179 degrees; 59.4 minutes; delta=0.6 minutes
SIGHTING for Sun-LL at 1981/3/27

Experiment to show that reasonably large variations on Dead Reckoning result in consistent fix positions.

In [14]:
data = [
    ["Altair", "1978/7/25 4:7:22", createAngle(44, 36, N), createAngle(122, 14, W), createAngle(30, 35.4, None), createAngle(0, 2, Off), 16],
    ["Antares", "1978/7/25 4:7:22", createAngle(44, 36, N), createAngle(122, 14, W), createAngle(18, 54.3, None), createAngle(0, 2, Off), 16],
]

delta = [
    [-30, -30],
    [-30, 30],
    [30, 30],
    [30, -30]
]

celestial_object, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt = data[0]
Ho_1, LHA_1, Hc_1, Zn_1, a_1, ta_1 = compute_position (celestial_object, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt)

celestial_object, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt = data[1]
Ho_2, LHA_2, Hc_2, Zn_2, a_2, ta_2 = compute_position (celestial_object, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt)

fix_lat_org, fix_lon_org = calculate_fix(dr_lat, dr_lon, Zn_1, ta_1, a_1, Zn_2, ta_2, a_2)
print("dr_lat: {} dr_lon: {}".format(angleToString(dr_lat), angleToString(dr_lon)))
print("Original Fix is Lat:{}; Lon:{}".format(angleToString(fix_lat_org), angleToString(fix_lon_org)))


for i in range (len(delta)):
    lat_delta, lon_delta = delta[i]
    lat_delta = createAngle(0, lat_delta, None) + dr_lat
    lon_delta = createAngle(0, lon_delta, None) + dr_lon
    
    celestial_object, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt = data[0]
    Ho_1, LHA_1, Hc_1, Zn_1, a_1, ta_1 = compute_position (celestial_object, utc, lat_delta, lon_delta, Hs, ic, heightEyeFt)

    celestial_object, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt = data[1]
    Ho_2, LHA_2, Hc_2, Zn_2, a_2, ta_2 = compute_position (celestial_object, utc, lat_delta, lon_delta, Hs, ic, heightEyeFt)

    fix_lat, fix_lon = calculate_fix(lat_delta, lon_delta, Zn_1, ta_1, a_1, Zn_2, ta_2, a_2)
    print("Variant: {} =================================".format(i))
    print("a_lat: {} a_lon: {}".format(angleToString(lat_delta), angleToString(lon_delta)))
    print("Fix is Lat:{}; Lon:{}".format(angleToString(fix_lat), angleToString(fix_lon)))
    print("Delta is Lat:{}; Lon:{}".format(angleToString(fix_lat-fix_lat_org), angleToString(fix_lon-fix_lon_org)))


dr_lat: 44 degrees; 36.0 minutes dr_lon: -122 degrees; 14.0 minutes
Original Fix is Lat:44 degrees; 39.07 minutes; Lon:-122 degrees; 12.62 minutes
Variant: 0 =================================
a_lat: 44 degrees; 6.0 minutes a_lon: -122 degrees; 44.0 minutes
Fix is Lat:44 degrees; 39.16 minutes; Lon:-122 degrees; 12.79 minutes
Delta is Lat:0 degrees; 0.09 minutes; Lon:0 degrees; 0.18 minutes
Variant: 1 =================================
a_lat: 44 degrees; 6.0 minutes a_lon: -121 degrees; 44.0 minutes
Fix is Lat:44 degrees; 39.14 minutes; Lon:-122 degrees; 12.69 minutes
Delta is Lat:0 degrees; 0.07 minutes; Lon:0 degrees; 0.07 minutes
Variant: 2 =================================
a_lat: 45 degrees; 6.0 minutes a_lon: -121 degrees; 44.0 minutes
Fix is Lat:44 degrees; 39.15 minutes; Lon:-122 degrees; 12.74 minutes
Delta is Lat:0 degrees; 0.08 minutes; Lon:0 degrees; 0.12 minutes
Variant: 3 =================================
a_lat: 45 degrees; 6.0 minutes a_lon: -122 degrees; 44.0 minutes
Fix i

Start far off real location and do two iterations to get to better fix.

In [15]:
data = [
    ["Altair", "1978/7/25 4:7:22", createAngle(44, 36, N), createAngle(122, 14, W), createAngle(30, 35.4, None), createAngle(0, 2, Off), 16],
    ["Antares", "1978/7/25 4:7:22", createAngle(44, 36, N), createAngle(122, 14, W), createAngle(18, 54.3, None), createAngle(0, 2, Off), 16],
]

celestial_object, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt = data[0]
Ho_1, LHA_1, Hc_1, Zn_1, a_1, ta_1 = compute_position (celestial_object, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt)

celestial_object, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt = data[1]
Ho_2, LHA_2, Hc_2, Zn_2, a_2, ta_2 = compute_position (celestial_object, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt)

fix_lat_org, fix_lon_org = calculate_fix(dr_lat, dr_lon, Zn_1, ta_1, a_1, Zn_2, ta_2, a_2)
print("dr_lat: {} dr_lon: {}".format(angleToString(dr_lat), angleToString(dr_lon)))
print("Original Fix is Lat:{}; Lon:{}".format(angleToString(fix_lat_org), angleToString(fix_lon_org)))

delta_degrees = -5

lat_delta = createAngle(delta_degrees, 0, None) + dr_lat
lon_delta = createAngle(delta_degrees, 0, None) + dr_lon

iterations = 2

for i in range (iterations):
    celestial_object, utc, _, _, Hs, ic, heightEyeFt = data[0]
    Ho_1, LHA_1, Hc_1, Zn_1, a_1, ta_1 = compute_position (celestial_object, utc, lat_delta, lon_delta, Hs, ic, heightEyeFt)

    celestial_object, utc, _, _, Hs, ic, heightEyeFt = data[1]
    Ho_2, LHA_2, Hc_2, Zn_2, a_2, ta_2 = compute_position (celestial_object, utc, lat_delta, lon_delta, Hs, ic, heightEyeFt)

    fix_lat, fix_lon = calculate_fix(lat_delta, lon_delta, Zn_1, ta_1, a_1, Zn_2, ta_2, a_2)
    print("Iteration: {} =================================".format(i))
    print("a_lat: {} a_lon: {}".format(angleToString(lat_delta), angleToString(lon_delta)))
    print("Fix is Lat:{}; Lon:{}".format(angleToString(fix_lat), angleToString(fix_lon)))
    print("Delta is Lat:{}; Lon:{}".format(angleToString(fix_lat-fix_lat_org), angleToString(fix_lon-fix_lon_org)))
    lat_delta = fix_lat
    lon_delta = fix_lon


dr_lat: 44 degrees; 36.0 minutes dr_lon: -122 degrees; 14.0 minutes
Original Fix is Lat:44 degrees; 39.07 minutes; Lon:-122 degrees; 12.62 minutes
Iteration: 0 =================================
a_lat: 39 degrees; 36.0 minutes a_lon: -127 degrees; 14.0 minutes
Fix is Lat:44 degrees; 47.65 minutes; Lon:-122 degrees; 25.36 minutes
Delta is Lat:0 degrees; 8.58 minutes; Lon:0 degrees; 12.74 minutes
Iteration: 1 =================================
a_lat: 44 degrees; 47.65 minutes a_lon: -122 degrees; 25.36 minutes
Fix is Lat:44 degrees; 39.09 minutes; Lon:-122 degrees; 12.62 minutes
Delta is Lat:0 degrees; 0.02 minutes; Lon:0 degrees; 0.0 minutes
